In [4]:
import pandas as pd
import numpy as np
import string
from bs4 import BeautifulSoup
import unidecode
from word2number import w2n
import contractions
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

from sklearn import metrics
from numpy import mean
from numpy import std


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics.pairwise import euclidean_distances
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif
from sklearn.feature_selection import VarianceThreshold

In [5]:
train_data = pd.read_csv("Train.csv", sep=',')
train_data.drop(train_data.filter(regex="Unnamed"), axis =1, inplace= True)
train_data.drop(train_data.filter(regex="id"), axis =1, inplace= True)
test_data = pd.read_csv("Test.csv", sep=',')
test_data.drop(train_data.filter(regex="Unnamed"), axis =1, inplace= True)
test_data.drop(train_data.filter(regex="id"), axis =1, inplace= True)

In [6]:
#remove URL and @xxx from text
def to_lower(text):
    text = text.lower()
    return text
def remove_url_and_at(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\S+", "", text)
    return text

# this fuction is from <https://gist.github.com/jiahao87/d57a2535c2ed7315390920ea9296d79f>
#remove extra whitespaces from text
def remove_whitespace(text):
    text = text.strip()
    return " ".join(text.split())

# this fuction is from <https://gist.github.com/jiahao87/d57a2535c2ed7315390920ea9296d79f>
#remove html tags from text
def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

# this fuction is from <https://gist.github.com/jiahao87/d57a2535c2ed7315390920ea9296d79f>
#remove accented characters from text
def remove_accented_chars(text):
    text = unidecode.unidecode(text)
    return text

# this fuction is from <https://gist.github.com/jiahao87/d57a2535c2ed7315390920ea9296d79f>
#expand contractions in text
def expand_contractions(text):
    #expand shortened words
    text = contractions.fix(text)
    return text

# standardize emoji
def emotion(text):
    text = re.sub(r':\)|:D|: \)|=\)|:-\)', " :) ", text)
    text = re.sub(r':\(|: \(|:-\(', " :(  ", text)                      
    return text

#remove punctuation beside !&?
def remove_punc_and_number(text):
    punctuation = list(string.punctuation)
    punctuation.remove('?')
    punctuation.remove('!')
    for i in range(len(text)):
        if text[i] in punctuation:
            if i+1 <= len(text)-1:
                if (text[i] == ":") and (text[i+1] == "(" or text[i+1] == ")"):
                    continue
            if i >= 1:
                if (text[i] == ")" or text[i] == "(") and (text[i-1] == ':'):
                    continue
            text = text.replace(text[i], ' ')
    text = re.sub(r'[0-9]+', '', text)
    return text


#extract !&?
def create_whitespace(text):
    text = re.sub(r'\?+', ' ? ', text)
    text = re.sub(r'!+', ' ! ', text)              
    return text

#remove words have <=2 length
def remove_shortwords(filtered_sentence):
    return_list=[]
    for word in filtered_sentence:
        if len(word) > 2:
            return_list.append(word)
    return  return_list

#lemitization
def lemmetize_print(text):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

# remove stop words from text
# exclude words "no","not",from stopwords list
stop_words = set(stopwords.words('english'))
stop_words.remove('not')
stop_words.remove('no')

def remove_stopwords(text):
    text_lst = text.split(" ")
    for word in text_lst:
        if word in stop_words:
            text_lst.remove(word)
    return text_lst

def remove_deplicated_charac(text):
    text = re.sub("(.)\\1{2,}", "\\1", text)
    return text

def remove_deplicated_charac2(text):
    text_lst = text.split(" ")
    for i in range(len(text_lst)):
        text_lst[i] = re.sub("^(.)\\1{1,}","\\1" ,text_lst[i])
    return " ".join(text_lst)

In [7]:
#preprocessing text
from collections import Counter
import pycountry


frequency = Counter()
month = ["january", "february", "march", "april", "june", "july", "august", "september", "october", "november","december"]
number =["one","two","three","four","five","six","seven","eight","nine","ten"]
countries= []
for p in pycountry.countries:
    p = p.name.lower()
    countries.append(p)
    
    
def clean_text_instance(df):
    for i in range(len(df["text"])):
        text = df["text"][i]
        text = to_lower(text)
        text = df["text"][i]
        text = contractions.fix(text) 
        text = remove_deplicated_charac(text)
        text = remove_deplicated_charac2(text)
        text = remove_accented_chars(text)
        text = remove_url_and_at(text)
        text = strip_html_tags(text)
        text = emotion(text)
        text = remove_punc_and_number(text)
        text = create_whitespace(text)
        text = remove_whitespace(text)
        text = lemmetize_print(text)

        #word_tokens = word_tokenize(text)
        stop_word_removed_list = remove_stopwords(text) 
        new_filtered_sentence = remove_shortwords(stop_word_removed_list)
        #record number of occurance for unique words
        frequency.update(new_filtered_sentence)
        new_sentence = " ".join(new_filtered_sentence)
        df.iloc[i,df.columns.get_loc('text')] = new_sentence
    #keep only words with more than 2 occurace
    for i in range(len(df["text"])):
        word_list = df["text"][i].split()
        new_word_list = [word for word in word_list if frequency[word] >3]
        new_word_list = [word for word in word_list if frequency[word] <7000]
        new_word_list = [word for word in word_list if word not in month]
        new_word_list = [word for word in word_list if word not in number]
        new_word_list = [word for word in word_list if word not in countries]
        new_sentence = " ".join(new_word_list)
        df.iloc[i,df.columns.get_loc('text')]= new_sentence 

In [8]:
clean_text_instance(test_data)
clean_text_instance(train_data)

In [9]:
#separating instance and label for Train
X_train_raw = [x[0] for x in train_data[['text']].values]
y_train_ori = [x[0] for x in train_data[['sentiment']].values]

#check the result
print("Train length:",len(X_train_raw))

#separating instance and label for Test
X_test_raw = [x[0] for x in test_data[['text']].values]

#check the result
print("Test length:",len(X_test_raw))

Train length: 21802
Test length: 6099


In [10]:
# split the training data into subtraining and validation set
X_train, X_val, y_train, y_val = train_test_split(X_train_raw, y_train_ori, test_size = 0.28, random_state = 42)

In the following blocks, either go with bow or tfidf, do not do both!

In [8]:
# vetorisation using BoW
from sklearn.feature_extraction.text import CountVectorizer
BoW_vectorizer = CountVectorizer(analyzer ='word', ngram_range =(1,2))
#Build the feature set (vocabulary) and vectorise the Tarin dataset using BoW
X_train_BoW = BoW_vectorizer.fit_transform(X_train)
#Use the feature set (vocabulary) from Train to vectorise the Test dataset 
X_val_BoW = BoW_vectorizer.transform(X_val)
print("Train feature space size a(using BoW):",X_train_BoW.shape)
print("Test feature space size (using BoW):",X_val_BoW.shape)
print(len(y_train))

Train feature space size a(using BoW): (15697, 132419)
Test feature space size (using BoW): (6105, 132419)
15697


In [24]:
# vetorisation using TFIDF
tfidf_vectorizer = TfidfVectorizer(analyzer ='word', ngram_range =(1,2))
#Build the feature set (vocabulary) and vectorise the Tarin dataset using TFIDF
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train,y_train)
#Use the feature set (vocabulary) from Train to vectorise the Test dataset 
X_val_tfidf = tfidf_vectorizer.transform(X_val)
print("Train feature space size (using BoW):",X_train_tfidf.shape)
print("Test feature space size (using BoW):",X_val_tfidf.shape)
print("Dimension of y_train:",len(y_train))

Train feature space size (using BoW): (15697, 132419)
Test feature space size (using BoW): (6105, 132419)
Dimension of y_train: 15697


In [12]:
#remove feature with low variance using bow
threshold = 0.0001
selector = VarianceThreshold(threshold  = threshold)
X_train_BoW_var = selector.fit_transform(X_train_BoW,y_train)
X_val_BoW_var = selector.transform(X_val_BoW)
print("Train feature space size (using BoW):",X_train_BoW_var.shape)
print("Test feature space size (using BoW):",X_val_BoW_var.shape)
print("Dimension of y_train:",len(y_train))

Train feature space size (using BoW): (15697, 21538)
Test feature space size (using BoW): (6105, 21538)
Dimension of y_train: 15697


In [25]:
# remove feature with low variance using tfidf
X_train_tfidf_var= selector.fit_transform(X_train_tfidf, y_train)
X_val_tfidf_var = selector.transform(X_val_tfidf)
print("Train feature space size (using tfidf):",X_train_tfidf_var.shape)
print("Test feature space size (using tfidf):",X_val_tfidf_var.shape)
print("Dimension of y_train:",len(y_train))

Train feature space size (using tfidf): (15697, 948)
Test feature space size (using tfidf): (6105, 948)
Dimension of y_train: 15697


In [13]:
#feature selection select K best feature using BoW
k_best1 = 5000
select_k_best = SelectKBest(f_classif, k = k_best1)
X_train_BoW_var_selectk = select_k_best.fit_transform(X_train_BoW_var, y_train)
X_val_BoW_var_selectk = select_k_best.transform(X_val_BoW_var)
print("Train feature space size (using BoW):",X_train_BoW_var_selectk.shape)
print("Test feature space size (using BoW):",X_val_BoW_var_selectk.shape)

Train feature space size (using BoW): (15697, 5000)
Test feature space size (using BoW): (6105, 5000)


In [26]:
#feature selection select K best feature using TFIDF
k_best2 = 948
select_k_best = SelectKBest(f_classif, k = k_best2)
X_train_tfidf_var_selectk = select_k_best.fit_transform(X_train_tfidf_var,y_train)
X_val_tfidf_var_selectk = select_k_best.transform(X_val_tfidf_var)

In [27]:
#oversampling
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train_BoW_var_selectk_over, y_train_BoW = oversample.fit_resample(X_train_BoW_var_selectk, y_train)
X_train_tfidf_var_selectk_over, y_train_tfidf = oversample.fit_resample(X_train_tfidf_var_selectk, y_train)

In [30]:
from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[0.1, 0.5, 1, 1.5, 2], "decision_function_shape": ("ovo", "ovr")}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_train_tfidf_var_selectk_over, y_train_tfidf)

print(clf.best_params_)

{'C': 2, 'decision_function_shape': 'ovo', 'kernel': 'rbf'}


In [18]:
from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from pandas import read_csv
from imblearn.over_sampling import SMOTE
from collections import Counter
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder

parameters2 = {'solver':('newton-cg', 'liblinear', 'sag', 'saga'), 'max_iter': [500, 2000], "C": [0.2, 0.4, 0.6, 0.8, 1, 1.5, 2]}
clf2 = GridSearchCV(LogisticRegression(), parameters2)
clf2.fit(X_train_tfidf_var_selectk_over, y_train_tfidf)

print(clf2.best_params_)

{'C': 1.5, 'max_iter': 500, 'solver': 'saga'}


In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
# Multinominal NB
parameters4 = {"alpha":[1,1.2,1.4,1.5,1.6,1.7,1.8,2],
              'fit_prior':[True, False]}
clf4 = GridSearchCV(estimator = MultinomialNB(), param_grid = parameters4)
clf4.fit(X_train_tfidf_var_selectk_over, y_train_tfidf)

print(clf4.best_params_)

{'alpha': 1.2, 'fit_prior': False}


In [29]:
from sklearn.neighbors import KNeighborsClassifier
parameters5 = {"n_neighbors":[5, 10, 20, 50, 100],
              'weights': ("uniform", "distance"),
              'algorithm': ('auto', 'brute')}
clf5 = GridSearchCV(estimator = KNeighborsClassifier(), param_grid = parameters5)
clf5.fit(X_train_tfidf_var_selectk_over, y_train_tfidf)

print(clf5.best_params_)

{'algorithm': 'auto', 'n_neighbors': 5, 'weights': 'distance'}
